<a href="https://colab.research.google.com/github/leakysam/merge-data-in-different-datasets/blob/main/merge_flows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load hubs data from CSV (contains country_id)
hubs_file = "/content/Hubs Updated without hub id site.csv"
hubs_df = pd.read_csv(hubs_file)

# Load country lookup data
country_file = "/content/csvjson.csv"  # replace with the actual path of the country lookup file
country_df = pd.read_csv(country_file)

# Load flows data from Excel
flows_file = "/content/FLOW MAP WORKING PERSONAL (Copy).xlsx"
flows_df = pd.read_excel(flows_file)

# Debugging: Check column names
print("Hubs columns:", hubs_df.columns)
print("Country columns:", country_df.columns)
print("Flows columns:", flows_df.columns)

# Strip column names to remove any spaces
hubs_df.rename(columns=lambda x: x.strip(), inplace=True)
country_df.rename(columns=lambda x: x.strip(), inplace=True)
flows_df.rename(columns=lambda x: x.strip(), inplace=True)

# Merge country names into hubs_df using country_id
hubs_df = hubs_df.merge(country_df, how="left", left_on="country_id", right_on="country_id").rename(
    columns={"name": "country_name"}  # Rename country to country_name
).drop(columns=["country_id"])

# Merge hubs information for `from_id`
flows_df = flows_df.merge(hubs_df, how="left", left_on="from_id", right_on="id").rename(
    columns={
        "name": "from_name",
        "country_name": "from_country",
        "coordinates": "from_coordinates"
    }
).drop(columns=["id"])

# Merge hubs information for `to_id`
flows_df = flows_df.merge(hubs_df, how="left", left_on="to_id", right_on="id").rename(
    columns={
        "name": "to_name",
        "country_name": "to_country",
        "coordinates": "to_coordinates"
    }
).drop(columns=["id"])

# Save the enriched flows file
flows_df.to_excel("flows_with_country_name.xlsx", index=False)

# Print the enriched flows DataFrame to verify
print(flows_df)


Hubs columns: Index(['id', 'name', 'country_id', 'coordinates', 'Unnamed: 4'], dtype='object')
Country columns: Index(['country_id', 'name', 'code', 'emoji', 'hasFlowMapData',
       'hasParitiesData', 'hasPriceData', 'region', 'currency', 'hasForexData',
       'subRegion', 'color', 'vat'],
      dtype='object')
Flows columns: Index(['hqw', 'lqw', 'vhp', 'br', 'year', 'flow_sum', 'from_id', 'to_id',
       'flow_type_id', 'Unnamed: 9'],
      dtype='object')
           hqw       lqw  vhp   br  year  flow_sum  from_id  to_id  \
0      34500.0       0.0  0.0  0.0  2020   34500.0       61      1   
1      15500.0       0.0  0.0  0.0  2020   15500.0      102      1   
2       7000.0       0.0  0.0  0.0  2020    7000.0        1      2   
3          0.0   43500.0  0.0  0.0  2020   43500.0       61      3   
4     238000.0  102500.0  0.0  0.0  2020  340500.0       79      3   
...        ...       ...  ...  ...   ...       ...      ...    ...   
3329    1500.0       NaN  NaN  NaN  2024      

In [ ]:
import pandas as pd

# Replace with your actual file path
file_path = "/content/flows_with_country_name.xlsx"

# Create DataFrame by reading the Excel file
df = pd.read_excel(file_path)

# Divide the flow sums by 12 (assuming the values in hqw, lqw, vhp, and br are annual totals)
df['hqw'] = df['hqw'] / 12
df['lqw'] = df['lqw'] / 12
df['vhp'] = df['vhp'] / 12
df['br'] = df['br'] / 12

# Create a 'Month' column with 12 months for each year
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Create an empty list to collect the final data
final_data = []

# Loop through each row in the DataFrame to create 12 rows for each entry (one for each month)
for _, row in df.iterrows():
    for month in months:
        # Create a dictionary for each row
        new_row = {
            'Year': row['year'],
            'Month': month,
            'Destination': row['name_y_y'],
            'Origin': row['name_y_x'],
            'Quantity': row[['hqw', 'lqw', 'vhp', 'br']].max(),  # Use the max of the flow values for Quantity
            'Grade': row[['hqw', 'lqw', 'vhp', 'br']].idxmax()  # Find the column name with the highest value for Grade
        }
        final_data.append(new_row)

# Create a new DataFrame with the formatted data
formatted_df = pd.DataFrame(final_data)

# Save the formatted DataFrame to an Excel file
formatted_df.to_excel("formatted_enriched_flows.xlsx", index=False)

# Display the final DataFrame
print(formatted_df)


       Year      Month Destination        Origin     Quantity Grade
0      2020    January     Eritrea        Brazil  2875.000000   hqw
1      2020   February     Eritrea        Brazil  2875.000000   hqw
2      2020      March     Eritrea        Brazil  2875.000000   hqw
3      2020      April     Eritrea        Brazil  2875.000000   hqw
4      2020        May     Eritrea        Brazil  2875.000000   hqw
...     ...        ...         ...           ...          ...   ...
40003  2024     August  Madagascar  South Africa   833.333333   hqw
40004  2024  September  Madagascar  South Africa   833.333333   hqw
40005  2024    October  Madagascar  South Africa   833.333333   hqw
40006  2024   November  Madagascar  South Africa   833.333333   hqw
40007  2024   December  Madagascar  South Africa   833.333333   hqw

[40008 rows x 6 columns]


In [ ]:
import pandas as pd

# Sample input data (replace with your actual file read if necessary)
data = "/content/formatted_enriched_flows.xlsx"

# Create DataFrame
df = pd.read_excel(data)

# Group by 'Year', 'Origin', 'Destination', and 'Grade' and sum the quantities
df_grouped = df.groupby(['Year', 'Origin', 'Destination', 'Grade'], as_index=False)['Quantity'].sum()

# Divide the summed quantities by 12 to get monthly values
df_grouped['Quantity'] = df_grouped['Quantity'] / 12

# Create the 'Month' column to repeat each row for each month of the year
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
expanded_data = []

for _, row in df_grouped.iterrows():
    for month in months:
        new_row = row.copy()
        new_row['Month'] = month
        expanded_data.append(new_row)

# Create a new DataFrame with the expanded data
final_df = pd.DataFrame(expanded_data)

# Save the formatted DataFrame to a CSV
final_df.to_excel("final_enriched_flows_with_months.xlsx", index=False)

# Display the final DataFrame
print(final_df)


      Year    Origin    Destination Grade     Quantity      Month
0     2018   Algeria         France   hqw  1593.166667    January
0     2018   Algeria         France   hqw  1593.166667   February
0     2018   Algeria         France   hqw  1593.166667      March
0     2018   Algeria         France   hqw  1593.166667      April
0     2018   Algeria         France   hqw  1593.166667        May
...    ...       ...            ...   ...          ...        ...
3162  2024  Zimbabwe  United States    br  1075.833333     August
3162  2024  Zimbabwe  United States    br  1075.833333  September
3162  2024  Zimbabwe  United States    br  1075.833333    October
3162  2024  Zimbabwe  United States    br  1075.833333   November
3162  2024  Zimbabwe  United States    br  1075.833333   December

[37956 rows x 6 columns]
